In [1]:
%reload_ext autoreload
%autoreload 2

import sys, os
import pickle

import numpy as np

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc_file('../matplotlibrc')

In [2]:
# import darkhistory
if os.environ['USER'] == 'yitians' and 'submit' in os.uname().nodename:
    os.environ['DH_DIR'] = '/work/submit/yitians/darkhistory/DarkHistory'
    os.environ['DM21CM_DATA_DIR'] = '/data/submit/yitians/DM21cm'
sys.path.append(os.environ['DH_DIR'])
# import DM21cm
sys.path.append('..')

In [3]:
from dm21cm.common import abscs_nBs_test

In [4]:
DATA_DIR = os.environ['DM21CM_DATA_DIR'] + '/transferfunctions/nBs_test'
os.listdir(DATA_DIR)

['phot_tfgv.npy',
 'phot_dep_ad.p',
 'phot_tf.interp',
 'phot_tf_ad.p',
 'phot_dep.interp',
 'phot_depgv.npy']

### 1. Energy deposition transfer function
- `tfgv` is f values ($E\rightarrow E$). To make it a $N\rightarrow E$ transfer function, need to first multiply by the energy abscissa.
- `tfgv` is ~ (nBs x rs Ein out). Transpose to make it ~ (rs Ein nBs x out).

In [5]:
abscs = {
    'rs' : abscs_nBs_test['rs'],
    'Ein' : abscs_nBs_test['photE'],
    'nBs' : abscs_nBs_test['nBs'],
    'x' : abscs_nBs_test['x'],
    'out' : abscs_nBs_test['dep_c']
}

In [6]:
abscs

{'rs': array([ 5.        ,  5.64418946,  6.37137493,  7.19224944,  8.1188837 ,
         9.16490355, 10.34569041, 11.67860735, 13.18325449, 14.88175721,
        16.79909143, 18.96345095, 21.40666199, 24.16465119, 27.27797391,
        30.79241055, 34.75963981, 39.23799852, 44.29333952, 50.        ]),
 'Ein': array([1.03931104e-04, 1.12262993e-04, 1.21262829e-04, 1.30984159e-04,
        1.41484823e-04, 1.52827298e-04, 1.65079070e-04, 1.78313035e-04,
        1.92607933e-04, 2.08048817e-04, 2.24727556e-04, 2.42743387e-04,
        2.62203501e-04, 2.83223683e-04, 3.05928998e-04, 3.30454541e-04,
        3.56946233e-04, 3.85561696e-04, 4.16471187e-04, 4.49858613e-04,
        4.85922622e-04, 5.24877791e-04, 5.66955896e-04, 6.12407295e-04,
        6.61502415e-04, 7.14533365e-04, 7.71815671e-04, 8.33690151e-04,
        9.00524950e-04, 9.72717723e-04, 1.05069801e-03, 1.13492977e-03,
        1.22591417e-03, 1.32419256e-03, 1.43034968e-03, 1.54501714e-03,
        1.66887720e-03, 1.80266680e-03, 1.947

In [7]:
data = np.load(DATA_DIR + '/phot_depgv.npy')
print(data.shape)
data_permuted = np.transpose(data, axes=[2, 3, 0, 1, 4])
print(data_permuted.shape)

(10, 10, 20, 500, 5)
(20, 500, 10, 10, 5)


In [8]:
# multiply in energy
eng = abscs['Ein']
data_NEtf = np.einsum('rEnxo,E->rEnxo', data_permuted, eng)
print(data_NEtf.shape)

(20, 500, 10, 10, 5)


In [9]:
pickle.dump((abscs, data_NEtf), open(DATA_DIR + '/phot_dep_ad.p', 'wb'))

### 2. Photon number transfer function
- `tfgv` is already in $N\rightarrow N$.
- `tfgv` is ~ (nBs x rs Ein out). Transpose to make it ~ (rs Ein nBs x out).

In [10]:
abscs = {
    'rs' : abscs_nBs_test['rs'],
    'Ein' : abscs_nBs_test['photE'],
    'nBs' : abscs_nBs_test['nBs'],
    'x' : abscs_nBs_test['x'],
    'out' : abscs_nBs_test['photE']
}

In [11]:
data = np.load(DATA_DIR + '/phot_tfgv.npy')
print(data.shape)
data_permuted = np.transpose(data, axes=[2, 3, 0, 1, 4])
print(data_permuted.shape)

(10, 10, 20, 500, 500)
(20, 500, 10, 10, 500)


In [12]:
pickle.dump((abscs, data_permuted), open(DATA_DIR + '/phot_tf_ad.p', 'wb'))